## rescale_time_stocks
* Convert 1 minute bar data from barchart to daily bar data

In [1]:
import numpy as np
import math
import pandas as pd
import datetime
from pandas.tools.plotting import autocorrelation_plot
%matplotlib inline
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.nan)
sys.path.append('./')
sys.path.append('../')
sys.path.append('../../')
import futures_series as futs


### Define folders (input and output)

In [75]:
DIR_MARKET_DATA = open('./temp_folder/market_data_folder.txt').read()
DIR_STOCK_DATA = f'{DIR_MARKET_DATA}/stocks'
DIR_FINAL_OUTPUT_DATA = './temp_folder/stocks/synth_daily'
try:
    os.makedirs(DIR_FINAL_OUTPUT_DATA)
    print(f'created {DIR_FINAL_OUTPUT_DATA}')
except:
    print(f'{DIR_FINAL_OUTPUT_DATA} already exists')
    pass

./temp_folder/stocks/synth_daily already exists


### Define stock list

In [76]:
stock_list = ['XLB','XLE','XLF','XLK','XLI','XLP','XLU','XLV','XLY','SPY']

### Iterate through the stock list, rescaling minute bars to daily bars

In [77]:
final_dict = {}
for stk in stock_list:
    df = pd.read_csv(f'{DIR_STOCK_DATA}/{stk}.csv')
    df2 = df.copy()
    df2['year'] = df.timestamp.apply(lambda t: int(t[0:4]))
    df2['month'] = df.timestamp.apply(lambda t: int(t[5:7]))
    df2['day'] = df.timestamp.apply(lambda t: int(t[8:10]))
    df2['hour'] = df.timestamp.apply(lambda t: int(t[11:13]))
    df2['min'] = df.timestamp.apply(lambda t: int(t[14:16]))
    allcols = ['year','month','day','open','close','high','low','volume']
    aggcols = ['year','month','day']
    agg_dict = {'close': 'mean',
     'high': 'max',
     'low': 'min',
     'open':'mean',
     'volume':'sum'}
    df2gb = df2[allcols].groupby(aggcols,as_index=False).agg(agg_dict)
    df2gb.year = df2gb.year.astype(int)
    df2gb.month = df2gb.month.astype(int)
    df2gb.day = df2gb.day.astype(int)
    df2gb['date'] = df2gb.apply(lambda r: datetime.datetime(int(r.year),int(r.month),int(r.day)).strftime('%d-%b-%y'),axis=1)
#     df2gb = df2gb.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})[['Date','Open','High','Low','Close','Volume']]
    final_dict[stk] = df2gb
    

FileNotFoundError: File b'/Users/bperlman1/dropbox/market_data/stocks/SPY.csv' does not exist

### Save the daily bars

In [48]:
for stk in final_dict.keys():
    df = final_dict[stk]
    df.to_csv(f'{DIR_FINAL_OUTPUT_DATA}/{stk}.csv',index=False)


### Create a DataFrame that combines the daily bars horizontally, so each stocks open, high,low,close,volume data is a feature named {stk}\_open,{stk}\_high,{stk}\_low,{stk}\_close,{stk}\_volume

In [71]:
for stk in final_dict.keys():
    df = final_dict[stk][['year','month','day','open','close','high','low','volume']]
    print(f'{stk},{df.year.min()},{df.year.max()},{len(df)}')

XLB,2008,2018,2677
XLC,2018,2018,129
XLE,2008,2018,2677
XLF,2008,2018,2677
XLK,2008,2018,2677
XLI,2008,2018,2677
XLP,2008,2018,2677
XLU,2008,2018,2676
XLV,2008,2018,2677
XLY,2008,2018,2677


In [73]:
final_dict['XLP'].head()

,year,month,day,close,high,low,open,volume,Date
0,2008,5,5,21.167320,21.20444,21.09910,21.167407,1225368,05-May-08
1,2008,5,6,21.130663,21.24207,21.03138,21.130615,961733,06-May-08
2,2008,5,7,21.167612,21.28722,21.00128,21.168048,2509162,07-May-08
3,2008,5,8,21.096200,21.15177,21.00880,21.096430,1717773,08-May-08
4,2008,5,9,20.931721,21.03890,20.72287,20.931153,2780696,09-May-08


In [70]:
df_allstks = None

for stk in final_dict.keys():
    df = final_dict[stk][['year','month','day','open','close','high','low','volume']]
    col_rename_dict = {c:f'{stk}_{c}' for c in ['open','close','high','low','volume'] }
    df = df.rename(columns=col_rename_dict)
    if len(df)<len(final_dict['XLK'])-10:
        continue
    print(len(df))
    if df_allstks is None:
        df_allstks = df.copy()
    else:
        df_allstks = df_allstks.merge(df,how='inner',on=['year','month','day'])

df_allstks.to_csv(f'{DIR_FINAL_OUTPUT_DATA}/all_stocks.csv',index=False)

2677
2677
2677
2677
2677
2677
2676
2677
2677


In [74]:
df_allstks.year.min()

2008